# Final Project
### Môn học: CSC17104 - Lập trình cho Khoa học dữ liệu
****

**Nhóm 26**
<table style='float:left;'>
    <tr>
        <th style='text-align:center;'>MSSV</th>
        <th style='text-align:center;'>Họ và Tên</th>
        <th style='text-align:center;'>Mức độ hoàn thành(%)</th>
        <th style='text-align:center;'>Nhiệm vụ</th>
    </tr>
    <tr>
        <td style='text-align:center;'>21120178</td>
        <td style='text-align:left;'>Văn Bá Bảo Huy</td>
        <td style='text-align:center;'>Mức độ hoàn thành</td>
        <td style='text-align:left;'>Nhiệm vụ</td>
    </tr>
    <tr>
        <td style='text-align:center;'>21120071</td>
        <td style='text-align:left;'>Nguyễn Thị Thanh Hoa</td>
        <td style='text-align:center;'>Mức độ hoàn thành</td>
        <td style='text-align:left;'>Nhiệm vụ</td>
    </tr>
    <tr>
        <td style='text-align:center;'>21120174</td>
        <td style='text-align:left;'>Nguyễn Thị Gái</td>
        <td style='text-align:center;'>Mức độ hoàn thành</td>
        <td style='text-align:left;'>Nhiệm vụ</td>
    </tr>
</table>

### Environment check & Import nescessary packages

In [1]:
import sys
sys.executable
!pip install -q kaggle

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from collections import Counter
import datetime
import statistics
import seaborn as sns
from matplotlib import cycler

# **Phase 1**: Collecting Data

**Dataset**: Movies Daily Update Dataset<br>
**Kaggle**: https://www.kaggle.com/datasets/akshaypawar7/millions-of-movies

In [ ]:
#Download data from Kaggle with api-key
notebook_dir = os.path.dirname(os.path.realpath("__file__"))
os.chdir(notebook_dir)
!mkdir kaggle
!touch kaggle/kaggle.json
api_json = {"username":"tyzenr","key":"08b25d2b177116a2cb7dfddc38ab964b"}
with open('kaggle/kaggle.json', 'w') as file:
    json.dump(api_json, file)
!chmod 600 kaggle/kaggle.json

with open('kaggle/kaggle.json', 'r') as file:
    api_json = json.load(file)

os.environ['KAGGLE_USERNAME'] = api_json['username']
os.environ['KAGGLE_KEY'] = api_json['key']

import kaggle
kaggle.api.authenticate()
kaggle.api.dataset_download_files('akshaypawar7/millions-of-movies', path='.', unzip=True)

**1. Dữ liệu về chủ đề gì? Nguồn dữ liệu?**<br>
- Chủ đề dữ liệu: Bộ dữ liệu của hơn 700.000 bộ phim trong tập dữ liệu TMDB.<br>
- Nguồn dữ liệu: Kaggle
<br>

**2. Kiểm tra vấn đề bản quyền của tập dữ liệu.**<br>
- `CC0: Public Domain` - No Right Reserved
- CC0 cho phép các nhà khoa học, nhà giáo dục, nghệ sĩ, những người sáng tạo và chủ sở hữu khác của nội dung được bảo vệ bản quyền hoặc cơ sở dữ liệu *từ bỏ* những lợi ích đó đối với tác phẩm của họ và do đó đặt chúng hoàn toàn nhất có thể trong phạm vi công cộng, để những người khác có thể tự do xây dựng, nâng cao và tái sử dụng tác phẩm cho bất kỳ mục đích nào mà không bị hạn chế theo luật bản quyền hoặc cơ sở dữ liệu.
<br>

**3. Tác giả của tập dữ liệu đã thu thập bằng cách nào?** <br>
- Dữ liệu được thu thập từ TMDB Open API.
<br>

# **Phase 2**: Exploring Data

### 2.0 Đọc dữ liệu

In [16]:
df = pd.read_csv("movies.csv")
df.head(3)

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02,129000000.0,352056482.0,116.0,Released,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-6...
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,65675816.0,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...
2,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06,200000000.0,407045464.0,127.0,Released,Unite or fall.,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,/woJbg7ZqidhpvqFGGMRhWQNoxwa.jpg,496450-569094-298618-385687-877100-598331-4628...


### 2.1 Có bao nhiêu hàng và bao nhiêu cột?

In [17]:
row, col = df.shape[0], df.shape[1]
print('Dữ liệu có', row, 'hàng và', col, 'cột.')

Dữ liệu có 722585 hàng và 20 cột.


###  2.2 Ý nghĩa của mỗi hàng là gì?

### 2.3 Có 'hàng trùng lặp' không?

### 2.4 Ý nghĩa của từng cột là gì?

In [18]:
df.columns

Index(['id', 'title', 'genres', 'original_language', 'overview', 'popularity',
       'production_companies', 'release_date', 'budget', 'revenue', 'runtime',
       'status', 'tagline', 'vote_average', 'vote_count', 'credits',
       'keywords', 'poster_path', 'backdrop_path', 'recommendations'],
      dtype='object')

1. `id`: TMDB id
2. `title`: Title của phim
3. `genres`: Thể loại phim
4. `original_language`: Ngôn ngữ của phim
5. `overview`: Mô tả ngắn về bộ phim
6. `popularity`:
7. `production_companies`: Công ty sản xuất
8. `release_date`: Ngày công chiếu
9. `budget`: Kinh phí thực hiện
10. `revenue`: Doanh thu phim
11. `runtime`: Thời lượng của bộ phim
12. `status`:  Tình trạng bộ phim (đã công chiếu, lên kế hoạch công chiếu, ...)
13. `tagline`:  Khẩu hiệu phim
14. `vote_average`: Điểm vote trung bình
15. `vote_count`: Số lượng vote
16. `credits`: Danh sách diễn viên
17. `keywords`: Từ khóa riêng biệt mô tả phim
18. `poster_path`: Đường dẫn đến poster phim
19. `backdrop_path`: Đường dẫn đến ảnh nền phim
20. `recommendations`: ID các phim được đề xuất

### 2.5 Kiểu dữ liệu hiện tại của mỗi cột là gì? Có cột nào có kiểu dữ liệu không phù hợp không?

### 2.6 Với mỗi cột số, các giá trị được phân bố như thế nào?
- Phần trăm giá trị còn thiếu là bao nhiêu?
- Tối thiểu? tối đa? Chúng có bất thường không?

### 2.7 Với mỗi cột phân loại, các giá trị được phân bổ như thế nào?
- Phần trăm giá trị còn thiếu là bao nhiêu?
- Có bao nhiêu giá trị khác nhau? Hiển thị một số giá trị đó.
- Chúng có bất thường không?

In [19]:
object_df = df.select_dtypes(include='object')
obj_col_df = pd.DataFrame( index=['missing_ratio', 'diff_value', 'values'], columns=object_df.columns)

obj_col_df.loc['missing_ratio'] = (object_df.isna().sum() / object_df.size) *100
obj_col_df.loc['diff_value']    = object_df.nunique()
obj_col_df.loc['values']        = [object_df[col].sample(min(5, obj_col_df.loc['diff_value'][col])).tolist() for col in object_df.columns]

obj_col_df

,title,genres,original_language,overview,production_companies,release_date,status,tagline,credits,keywords,poster_path,backdrop_path,recommendations
missing_ratio,0.000064,2.242164,0.0,1.260826,4.102238,0.553515,0.0,6.539726,2.394608,5.452262,1.968233,5.320258,7.321992
diff_value,575337,11033,167,541712,147231,41303,6,95664,422998,119486,482846,196852,30806
values,"[How to Re-Caulk Your Tub, Noah & Annie, Les E...","[nan, Comedy-Music, nan, nan, Documentary]","[es, en, ko, nl, en]",[A story about a mountain man and his struggle...,"[Large Door Ltd.-Channel 4 Television, Davi Pr...","[1994-04-02, 2018-02-14, 2016-06-16, 1976-01-1...","[Released, Released, Released, Released, Relea...","[nan, nan, nan, 1, nan]",[Felix Dormann-Eugenie Bernay-Heinrich Eisenba...,"[nan, nan, nan, nan, nan]","[/zzSpbt1lrply3W4m4QmNxQJTgjn.jpg, /4ljYZ5H1jF...","[nan, nan, /o3MKS1awt0A9vKluUtiZaAqqHTt.jpg, n...","[nan, 791155-960704-772973-884363-1024530-1023..."


# **Phase 3**: Preprocessing & Asking Meaningful questions
Yêu cầu:
- Số lượng câu hỏi đặt ra cần lớn hơn hoặc bằng số lượng thành viên ($\ge 3$)
- Câu hỏi đặt ra cần có ý nghĩa (tác dụng của việc tìm kiếm câu trả lời cho câu hỏi đó) và không được quá dễ 🙂
- Chú ý đến chất lượng của từng câu hỏi, không phải số lượng câu hỏi.
- Trong file Notebook, với mỗi câu hỏi cần thể hiện được:
    - Nội dung câu hỏi
    - Ý nghĩa của câu hỏi

## **3.0** Preprocessing

## **3.1** Asking meaningful question

<table style='float:left;'>
    <tr>
        <th style='text-align:center;'>STT</th>
        <th style='text-align:center;'>Nội dung</th>
        <th style='text-align:center;'>Trình bày</th>
    </tr>
    <tr>
        <td style='text-align:center;'>Câu 1</td>
        <td style='text-align:left;'>Nội dung</td>
        <td style='text-align:center;'><a href='#cauhoi_1'>C1</a></td>
    </tr>
    <tr>
        <td style='text-align:center;'>Câu 2</td>
        <td style='text-align:left;'>Nội dung</td>
        <td style='text-align:center;'><a href='#cauhoi_2'>C2</a></td>
    </tr>
    <tr>
        <td style='text-align:center;'>Câu 3</td>
        <td style='text-align:left;'>Nội dung</td>
        <td style='text-align:center;'><a href='#cauhoi_3'>C3</a></td>
    </tr>
</table>

# **Phase 4**: Analyzing data to answer each question

### **Câu hỏi 1**: ghi nội dung ở đây<a name="cauhoi_1"></a>

**Ý nghĩa:**

### **Câu hỏi 2**: ghi nội dung ở đây<a name="cauhoi_2"></a>

**Ý nghĩa:**

### **Câu hỏi 3**: ghi nội dung ở đây<a name="cauhoi_3"></a>

**Ý nghĩa:**

# **Phase 5**: Reflection

## 5.0 Team members

**Văn Bá Bảo Huy**
- Khó khăn:
- Điều đã học được:

**Nguyễn Thị Gái**
- Khó khăn:
- Điều đã học được:

**Nguyễn Thị Thanh Hoa**
- Khó khăn:
- Điều đã học được:

## 5.1 Team

Nếu có nhiều thời gian hơn, nhóm sẽ ...